In [ ]:
from datetime import datetime
# get current date and year
now = datetime.now()

date = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y")
print(date)
time = now.strftime("%H_%M")
print("time:", time)

# Sparse components (sparse dictionary) analysis on multiple expressions meshes
## Methods
    - Sparse PCA
    - MiniBatchSparsePCA

## Process
- READ OBJs systematically
    - Make them functions to be called later
- Perform different components analysis methods
    - Select method
    - Take into account only vertices which represebts the front face region which effectively used for facial expressions
- Save in .pkl (which contains datastruct_blendshape)

## requirements/test-environment
- Manjaro Linux
- python 3.12.0
- Anaconda 22.9.0
    - packages-list
        - see requirement.txt

# Read OBJs
- place downloaded tracked meshes of multiple expressions /dataset/multiface/tracked_mesh/

In [ ]:
import sys
import os
# set path to dataset
path_to_dataset = os.path.join(os.getcwd(), '../dataset/multiface/tracked_mesh/')
save_path = "../dataset/multiface/tracked_mesh"

ID = 6795937

In [ ]:
# create a list of expressions
list_exps_name = []
map_exps2id = {}
counter = 0
for i, name in enumerate(os.listdir(path_to_dataset)):
    f = os.path.join(path_to_dataset, name)
    if os.path.isdir(f) and name.startswith('E0'):
        counter = counter + 1
        list_exps_name.append(name)


list_exps_name.sort()

for i, exp_name in enumerate(list_exps_name):
    print(f'{i}, {exp_name}')
    map_exps2id.update({exp_name: i})




Read OBJ in ./dataset/multiface

In [ ]:
# list of expressions
print(list_exps_name)
print(map_exps2id)

File handler for .obj in ./utils/FolderHandler.py

In [ ]:
from utils.Dataset_handler import Filehandler

In [ ]:
file_handler = Filehandler(path_to_dataset=path_to_dataset)
print(file_handler.get_path_to_dataset())
file_handler.iter_dir()
print("Expressions: Number of tracked mesh")
for key in file_handler.dict_objs.keys():
    print(f'{list_exps_name[key]}: {len(file_handler.dict_objs[key])}')
    print(file_handler.dict_objs[key]) 
# print(file_handler.dict_objs)

OBJ class

In [ ]:
from utils.OBJ_helper import OBJ

Test .obj loading

In [ ]:
test_path = "../dataset/multiface/tracked_mesh/E003_Neutral_Eyes_Closed/000783.obj"
test_obj = OBJ(test_path, swapyz=False)
print(f'Number of vertices: {len(test_obj.vertices)}')
print(test_obj.vertices)

load all .obj in the certain directory

In [ ]:
dict_expOBJs = {}
dict_expVerts = {}

# select number of samples for a expression 
num_samples_perExp = 20

for expID, key in enumerate(file_handler.dict_objs.keys()):
    list_OBJs = []
    list_Verts = []

    # since there are many sequences for a expression, we assume that second half of tracked mesh in a sequence captured the specific expressions
    # We only use second half of sequence for a expression.
    # This is also for resonable memory usage as well. if you run over all, you will consume more than 30GB memory to store all of objects

    # half_id = int(len(file_handler.dict_objs[key])/2)
    # end_id = int(len(file_handler.dict_objs[key]))

    for i, obj in enumerate(file_handler.dict_objs[key][0:num_samples_perExp]):
        path_to_obj = os.path.join(file_handler.list_expPathFiles[expID], obj)
        # print(path_to_obj)
        obj = OBJ(path_to_obj, swapyz=False)
        list_OBJs.append(obj)
        list_Verts.append(obj.vertices)
    dict_expOBJs.update({expID: list_OBJs})
    dict_expVerts.update({expID: list_Verts})

check the number of objs

In [ ]:
# for key in dict_expVerts.keys():
#     vertices = dict_expVerts[key]
#     print(len(vertices))

Test conversion from list to numpy array

In [ ]:
exp0_vertices = dict_expVerts[0]
print(f"Expression: {list_exps_name[0]}")
print(f"The number of meshes: {len(exp0_vertices)}")
print(f"The number of vertices for each mesh: {len(exp0_vertices[0])}")

In [ ]:
import numpy as np
import pandas as pd
num_vertices = 7306
len_col = num_vertices*3
print(f"The number of vertices for each meesh: {num_vertices}")
print(f"The number of columns of matrix X: {len_col}")

Test concatenation and check it

In [ ]:
_np_array = np.array(exp0_vertices)
print(_np_array.shape)
print(_np_array[0][0][0:3])

In [ ]:
_array = np.array(exp0_vertices).reshape((_np_array.shape[0], len_col))
print(_array.shape)
print(_array[0][0:3])

Concatenation of vertex positions for all tracked meshes of 5 expressions

In [ ]:
_list_xs = []
num_sum_samples = 0
for key in dict_expVerts.keys():
    # exclulde  Neutral Eyes Open (Since it could be similar to averafe face of target person)
    vertices = dict_expVerts[key]
    _num_samples = len(vertices)
    print(_num_samples)
    num_sum_samples = num_sum_samples + _num_samples
    _array = np.array(vertices).reshape((_num_samples, len_col))
    _list_xs.append(_array)

# print(f"Len of _list_xs: {len(_list_xs)}")
# print(f"The number of samples: {num_sum_samples}")

In [ ]:
neutralmesh_verts = _list_xs[0]
X = _list_xs[0]
for x in _list_xs[1:]:
    X = np.concatenate((X, x), axis = 0)
    # print(X)
print(X.shape)


Obtain the mean vertex coordinates over the sequence in "E001_Neutral_Eyes_Opens"
- We assume that the average vertex coordinate over the sequence in "E001_Neutral_Eyes_Open" defines the ID's neutral face mesh

In [ ]:
ave_neutralmesh_vertices = np.mean(neutralmesh_verts, axis = 0)
print(ave_neutralmesh_vertices.shape)

For PCA, we need to centrailzed data at the neutral face which is defined like above not at the average expression
- We need to subtract the matrix from the average vertex coordinates of the sequence of neutral face ("E001_Neutral_Eyes_Opened")

In [ ]:
cent_X = X-ave_neutralmesh_vertices[None, :]

Look at the contents in matrix X

In [ ]:
# df = pd.DataFrame(X)
# print(df)

# Face mask
- Masking the vertices to take into account only vertices composing the front face as you see below in blue

- To do it, we only perform over these blue vertices.
![face_mask_region](../images/Facemask_side01.png)

- Approach
    - In the data matrix, we replace vertices coordinate with 0 over the vertices composing other region 

In [ ]:
from utils.Blendshape import FaceMask
from utils.pickel_io import dump_pckl, load_from_memory

# set the name of pickel file to be loaded
pickel_fname = "FaceMask_30102023_09_40.pkl"

facemask = load_from_memory(path_to_memory = save_path, pickle_fname = pickel_fname)


In [ ]:
masked_cent_X = cent_X * facemask.bit_mask[None, :]

dump data matrix with mean (average neutral face)

In [ ]:
# dump the data matrix if you need to dump the matrix to save loading time
from utils.Blendshape import ZeroMeanDefMatrix
deformation_data = ZeroMeanDefMatrix(masked_cent_x = masked_cent_X, mean = ave_neutralmesh_vertices)
dd_pickel_fname = 'deformation_data_matrix_and_mean'+ '_' +date+'_'+time+'.pkl'
dump_pckl(data = deformation_data, save_root= save_path, pickel_fname=dd_pickel_fname)

In [ ]:
# check if the data matrix is masked
# we chose the ID:3567 which is located on top of nose, this vertices should be masked with 1
# as the center of facemesh which is obtained by the nearest neighboring search at the vertex

print(sum(masked_cent_X[:,3567*3]) != 0)
print(sum(masked_cent_X[:,3567*3+1]) != 0)
print(sum(masked_cent_X[:,3567*3+2]) != 0)

## Component Analysis on tracked meshes
- Select method from below
    - SparsePCA
    - MiniBatchSparsePCA

In [ ]:
# name_method = 'SparsePCA'
name_method = 'MiniBatchSparsePCA'
ESTIMATOR = None

# number of components
# if _n_components == None, the configuration of the number of components is by default
_n_components = 100

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# configure the esitimator as you need

if name_method == 'SparsePCA':
    try:
        from sklearn.decomposition import SparsePCA
        ESTIMATOR = SparsePCA(n_components = _n_components, max_iter = 100, verbose = True)
    except ImportError:
        print("Import Error: sklearn.decomposition.SparsePCA")
elif name_method == 'MiniBatchSparsePCA':
    try:
        from sklearn.decomposition import MiniBatchSparsePCA
        ESTIMATOR = MiniBatchSparsePCA(n_components=_n_components, verbose = True)
    except ImportError:
        print("Import Error: sklearn.decomposition.MiniBatchSparsePCA")

# Find the set of components
- SparsePCA (scikit-learn, this is used dictionary learning)
    - Find the set of sparse components that can optimally reconstruct the data.
    $$(U*, V*) = argmin_{U, V} \frac{1}{2} ||X - UV||_{F}^2 + \alpha||V||_{1, 1}$$
    $$ \text{subject to } ||U_k||_2 \leq 1 \text{, for all } k = {0,...,n_{components}}$$
- MiniBatchSparsePCA (scikit-learn, this is used MiniBatchDicionaryLearning)
    - Online learning, iterating over batches of the set of features (vertices), for a given number of iterations
    - This is faster but less accurate than SparsePCA

In [ ]:
est = ESTIMATOR.fit(masked_cent_X)

In [ ]:
C = None #each row represents kth component (should be sparse)
W = None #each column represents a weight for kth component
if name_method == 'SparsePCA' or name_method == 'MiniBatchSparsePCA':
     C = est.components_ #right-hand side V
     W = est.transform(masked_cent_X) #left-hand side U
# elif name_method == 'DictionaryLearning' or name_method == 'MiniBatchDictionaryLearning':
#      _dict = est.components_ #V
#      _code = est.transform(masked_cent_X) #U
#      C = _code.T
#      W = _dict.T
MEAN =  ave_neutralmesh_vertices # neutral face mesh (vertex coodinates of neutral face)

In [ ]:
# Sparsity check
# close to 1: Sparse, close to 0: Dense
sparsity_level = np.mean(C==0)
print(sparsity_level)

In [ ]:
print(f"Shape of C (right-handside): {C.shape}")
print(f"Shape of W (left-handside): {W.shape}")
print(f"Shape of MEAN (average expressions): {MEAN.shape}")

In [ ]:
max_collection_W = np.max(W, axis=0)
std_W = np.std(W, axis=0)

In [ ]:
print(max_collection_W.shape)
print(std_W.shape)

# Visulization of the weight matrix
- This is corresponding to the transposed dictionary
- Each color represents atom in the dictionary
  - This can be interpreted as the projected coordinate on the corresponding axis in the new coordinate system in C
- Each row represents K-latent factor vector of the certain tracked mesh data
- A vector should be estimated to generate a new expression

In [ ]:
Wx_data = np.arange(0, W.shape[0],1)
Wy_data = W
plt.plot(Wx_data, Wy_data)
plt.xlabel("Components")
plt.ylabel("Value")
plt.title("Coefficient (row) vectors in matrix U")
plt.show()

# Visualization of the sparse component matrix C
- This is corresponding to transposed code
- Each color represents new single axis in the new coordinate system

In [ ]:
x_data = np.arange(0,C.shape[1], 1)
y_data = C.T
plt.plot(x_data, y_data[:,:3])
plt.xlabel("Vertices")
plt.ylabel("Displacement")
plt.title("Components in matrix V (3 components from first row)")
plt.show()

# Reconstruction error of the sparse coded signal

In [ ]:
reconstructed_masked_cent_X = W @ C
error_rate = np.mean(np.sum((reconstructed_masked_cent_X - masked_cent_X)**2, axis = 1) / np.sum(masked_cent_X**2, axis = 1))
print(f"The reconstruction error of the sparse coded signal relative to the squared euclidean norm of the original signal: {error_rate}")

# Create a new expressions using sparse components

In [ ]:
# select D
D_Pcs = _n_components

In [ ]:
coefficients = []
Stds = []
for i in range(D_Pcs):
    mu,sigma = 0, std_W[i] # mean and standard deviation
    Stds.append(sigma)
    _noise = np.random.normal(mu, sigma, 1) + 0.5
    coefficients.append(abs(_noise))

In [ ]:
print(coefficients)
print(len(coefficients))

add linear combination of principal components, in which are weighted by coefficients to mean vertex position

In [ ]:
newFace_vertices = MEAN
for coeff in coefficients:
    _item = coeff*C[i]
    newFace_vertices = newFace_vertices + _item

In [ ]:
print(newFace_vertices)
print(newFace_vertices.shape)
print(newFace_vertices.shape[0]/3)

write a .obj based on obtained vertex positions for visualization
- Since the tracked mesh are topologically equivalent, we can easily obtain .obj only by rewriting line for vertex position (starting with 'v')

In [ ]:
# path configuration
sample_path = "../dataset/multiface/tracked_mesh/sample.obj"

ourput pointclouds (.obj) based on obtained vertex coordinates
>>check /dataset/multiface/tracked_mesh/result_point_clouds.obj

In [ ]:
# Sample obj file should be read first
obj = OBJ(sample_path, swapyz=False)
obj.write_PointClouds(save_path, newFace_vertices, mesh_name = "result_point_clouds_SparsePCA")

ourput mesh (.obj) based on obtained vertex coordinates
>>check /dataset/multiface/tracked_mesh/result_mesh.obj

In [ ]:
# Sample obj file should be read first
OBJ.write_OBJfile(reference_obj_file=sample_path, save_path=save_path, vertices=newFace_vertices, name_Exp="allPcs_SparsePCA")

# Interpretation of each blendshape
- Output each blendshape
- Compare with average face

In [ ]:
from utils.Blendshape import Blendshape

obtain each blenshape (.obj)

In [ ]:
blendshapes = []

if not os.path.exists(os.path.join(save_path, "First10Pcs_SparsePCA_test")):
    os.mkdir(os.path.join(save_path, "First10Pcs_SparsePCA_test"))


OBJ.write_OBJfile(reference_obj_file= sample_path, save_path = save_path, vertices=MEAN, name_Exp="averageExp")

for i in range(1, 10):
    _blendshape = Blendshape(Verts_averageExp=MEAN, PCs=C, Stds = Stds, D = i, save_path=os.path.join(save_path, "First10Pcs_SparsePCA_test"), only_specific_pc=True, name_newExp=str(i))
    _blendshape.sample_coefficients()
    _blendshape.get_newExp()
    fname = _blendshape.generate_newExp()
    blendshapes.append(_blendshape)

dump PCs, Stds, MEAN in .pickel for visualization

In [ ]:
from utils.Blendshape import datastruct_blendshape

print(type(MEAN))
print(type(C))
print(type(Stds))
print(save_path)

In [ ]:
our_blendshape = datastruct_blendshape(ID = ID, List_exps = list_exps_name, MEAN = MEAN, PCs = C, Stds = Stds)
# print(our_blendshape)
pickel_fname = 'blendshape_SparsePCA_'+date+'_'+time+'.pkl'
# dump datastruct_blendshape
dump_pckl(data=our_blendshape, save_root=save_path, pickel_fname=pickel_fname)

In [ ]:
print(our_blendshape)

load blendshape from memory (pickel)

In [ ]:
load_from_memory(path_to_memory=save_path, pickle_fname=pickel_fname)